In [1]:
import os, random, numpy as np, tensorflow as tf
os.environ["PYTHONHASHSEED"]="137";
random.seed(137);
np.random.seed(137);
tf.random.set_seed(137)


2025-09-07 09:03:23.251227: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-07 09:03:23.316619: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-09-07 09:03:23.316669: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-09-07 09:03:23.316716: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-07 09:03:23.327590: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-07 09:03:24.927139: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT

In [2]:
!elapsed --check

/bin/bash: line 1: elapsed: command not found


In [7]:
import os, sys
from pathlib import Path
if "TAGDIR" not in os.environ:
  os.environ["TAGDIR"] = str(Path.cwd().parent)
  tagdir = Path(os.environ["TAGDIR"]).resolve()
  (outputs := tagdir/"outputs").mkdir(parents=True, exist_ok=True)
  (tagdir/"outputs"/"csv_logs").mkdir(parents=True, exist_ok=True)
else:
  tagdir = Path(os.environ["TAGDIR"]).resolve()
for p in (tagdir, tagdir.parent):
  sp = str(p)
  if sp not in sys.path:
    sys.path.insert(0, sp)
os.environ.setdefault("OMP_NUM_THREADS","4")
os.environ.setdefault("TF_NUM_INTEROP_THREADS","2")
os.environ.setdefault("TF_NUM_INTRAOP_THREADS","4")
os.environ.setdefault("CUDA_VISIBLE_DEVICES","")
print("TAGDIR=", tagdir)

TAGDIR= /home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_03_e2e


In [5]:
"TAGDIR" not in os.environ

False

In [9]:
# Train/evaluate/log (baseline)

import tensorflow as tf
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train.astype("float32")/255.0
x_test = x_test.astype("float32")/255.0
from tensorflow.keras import layers, models
m = models.Sequential([
  layers.Input((32,32,3)),
  layers.Conv2D(32,3,activation="relu"),
  layers.Conv2D(32,3,activation="relu"),
  layers.MaxPooling2D(),
  layers.Conv2D(64,3,activation="relu"),
  layers.Conv2D(64,3,activation="relu"),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation="relu"),
  layers.Dense(10, activation="softmax"),
])
m.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
m.fit(x_train, y_train, epochs=5, batch_size=128, validation_split=0.1, verbose=1)
loss, acc = m.evaluate(x_test, y_test, verbose=0)
print(f"[DONE] test_acc={acc:.4f}")
from scripts.py_utils_p_03_e2e import log_test_summary
log_test_summary(acc, loss=float(loss), seed=137, tagdir=os.environ["TAGDIR"])

2025-09-07 09:14:04.008583: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 552960000 exceeds 10% of free system memory.


Epoch 1/5
  1/352 [..............................] - ETA: 16:37 - loss: 2.3117 - accuracy: 0.0625

2025-09-07 09:14:17.442781: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 27998208 exceeds 10% of free system memory.
2025-09-07 09:14:17.447657: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 27998208 exceeds 10% of free system memory.
2025-09-07 09:14:17.947457: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 27998208 exceeds 10% of free system memory.
2025-09-07 09:14:17.953139: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 27998208 exceeds 10% of free system memory.


352/352 [==============================] - 179s 500ms/step - loss: 1.6201 - accuracy: 0.4102 - val_loss: 1.2937 - val_accuracy: 0.5422
Epoch 2/5
352/352 [==============================] - 174s 493ms/step - loss: 1.2498 - accuracy: 0.5528 - val_loss: 1.1626 - val_accuracy: 0.5910
Epoch 3/5
352/352 [==============================] - 187s 532ms/step - loss: 1.0759 - accuracy: 0.6209 - val_loss: 1.1021 - val_accuracy: 0.6134
Epoch 4/5
352/352 [==============================] - 178s 505ms/step - loss: 0.9589 - accuracy: 0.6628 - val_loss: 0.9323 - val_accuracy: 0.6800
Epoch 5/5
352/352 [==============================] - 171s 485ms/step - loss: 0.8537 - accuracy: 0.7013 - val_loss: 0.8888 - val_accuracy: 0.6896
[DONE] test_acc=0.6780


ImportError: cannot import name 'log_test_summary' from 'scripts.py_utils_p_03_e2e' (/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_03_e2e/scripts/py_utils_p_03_e2e.py)

In [10]:
# Try to import; if the function isn't there yet, define a minimal fallback.
try:
    from scripts.py_utils_p_03_e2e import log_test_summary
except (ImportError, AttributeError):
    import json, csv, os, time
    from pathlib import Path
    def log_test_summary(acc: float, loss: float, seed: int, tagdir: str):
        td = Path(tagdir).resolve()
        out = td / "outputs"
        out.mkdir(parents=True, exist_ok=True)
        ts = time.strftime("%Y%m%d_%H%M%S")

        # JSON
        js = {"timestamp": ts, "seed": seed, "test_acc": float(acc), "loss": float(loss)}
        with open(out / f"test_summary_seed{seed}_{ts}.json", "w", encoding="utf-8") as f:
            json.dump(js, f, ensure_ascii=False, indent=2)

        # CSV
        csv_path = out / f"csv_logs/train_history_seed{seed}_{ts}.csv"
        csv_path.parent.mkdir(parents=True, exist_ok=True)
        with open(csv_path, "w", newline="", encoding="utf-8") as f:
            w = csv.writer(f)
            w.writerow(["timestamp","seed","metric","value"])
            w.writerow([ts, seed, "test_acc", float(acc)])
            w.writerow([ts, seed, "loss", float(loss)])

        print(f"[DONE] test_acc={acc:.4f}  | JSON & CSV written to {out}")

In [11]:
import os
log_test_summary(acc, loss=float(loss), seed=137, tagdir=os.environ["TAGDIR"])

[DONE] test_acc=0.6780  | JSON & CSV written to /home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_03_e2e/outputs


In [12]:
!pwd

/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_03_e2e/notebooks


In [13]:
!cd ..

In [14]:
!pwd

/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_03_e2e/notebooks


In [15]:
# I'll just have to check stuff on the terminal

```bash
=====..........................................................retval=0.........

(vanilla_cnn)  <=> conda environment, blank means none activated
[cifar10-vanilla-cnn] <=> git branch, blank means not in a git repo
bballdave025@MY-MACHINE ~/my_repos_dwb/fhtw-paper-code-prep
$ cd test_project_bash/p_03_e2e/
=====..........................................................retval=0.........

(vanilla_cnn)  <=> conda environment, blank means none activated
[cifar10-vanilla-cnn] <=> git branch, blank means not in a git repo
bballdave025@MY-MACHINE ~/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_03_e2e
$ ls -1 outputs/
csv_logs
gradcam_images
test_summary_seed137_20250907_095057.json
=====..........................................................retval=0.........

(vanilla_cnn)  <=> conda environment, blank means none activated
[cifar10-vanilla-cnn] <=> git branch, blank means not in a git repo
bballdave025@MY-MACHINE ~/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_03_e2e
$ ls -1 outputs/csv_logs
train_history_seed137_20250907_095057.csv
=====..........................................................retval=0.........

(vanilla_cnn)  <=> conda environment, blank means none activated
[cifar10-vanilla-cnn] <=> git branch, blank means not in a git repo
bballdave025@MY-MACHINE ~/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_03_e2e
$ wc -l < outputs/test_summary_seed137_20250907_095057.json 
5
=====..........................................................retval=0.........

(vanilla_cnn)  <=> conda environment, blank means none activated
[cifar10-vanilla-cnn] <=> git branch, blank means not in a git repo
bballdave025@MY-MACHINE ~/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_03_e2e
$ cat outputs/test_summary_seed137_20250907_095057.json 
{
  "timestamp": "20250907_095057",
  "seed": 137,
  "test_acc": 0.6779999732971191,
  "loss": 0.9174857139587402
}=====..........................................................retval=0.........

(vanilla_cnn)  <=> conda environment, blank means none activated
[cifar10-vanilla-cnn] <=> git branch, blank means not in a git repo
bballdave025@MY-MACHINE ~/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_03_e2e
$ wc -l < outputs/csv_logs/train_history_seed137_20250907_095057.csv 
3
=====..........................................................retval=0.........

(vanilla_cnn)  <=> conda environment, blank means none activated
[cifar10-vanilla-cnn] <=> git branch, blank means not in a git repo
bballdave025@MY-MACHINE ~/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_03_e2e
$ cat outputs/csv_logs/train_history_seed137_20250907_095057.csv 
timestamp,seed,metric,value
20250907_095057,137,test_acc,0.6779999732971191
20250907_095057,137,loss,0.9174857139587402
=====..........................................................retval=0.........
(vanilla_cnn)  <=> conda environment, blank means none activated
[cifar10-vanilla-cnn] <=> git branch, blank means not in a git repo
bballdave025@MY-MACHINE ~/my_repos_dwb/fhtw-paper-code-prep
$ elapsed --check
3882
=====..........................................................retval=0.........

(vanilla_cnn)  <=> conda environment, blank means none activated
[cifar10-vanilla-cnn] <=> git branch, blank means not in a git repo
bballdave025@MY-MACHINE ~/my_repos_dwb/fhtw-paper-code-prep
$ echo "$(elapsed --check) / 60" | bc -l
64.95000000000000000000
=====..........................................................retval=0.........

```

### About an hour isn't too bad,

especially considering the training time on a CPU. We had things
training by 45 minutes. I'm going to `cat` a lot of the scripts
here, just to have backup in another place.

## Scripts

### `bash` scripts

```bash
(vanilla_cnn)  <=> conda environment, blank means none activated
[cifar10-vanilla-cnn] <=> git branch, blank means not in a git repo
bballdave025@MY-MACHINE ~/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_03_e2e
$ cd ../../../
=====..........................................................retval=0.........

(vanilla_cnn)  <=> conda environment, blank means none activated
 <=> git branch, blank means not in a git repo
bballdave025@MY-MACHINE ~/my_repos_dwb
$ cd fhtw-paper-code-prep/
=====..........................................................retval=0.........

(vanilla_cnn)  <=> conda environment, blank means none activated
[cifar10-vanilla-cnn] <=> git branch, blank means not in a git repo
bballdave025@MY-MACHINE ~/my_repos_dwb/fhtw-paper-code-prep
$ find . -type f -iname "*.sh" | wc -l
17
=====..........................................................retval=0.........

(vanilla_cnn)  <=> conda environment, blank means none activated
[cifar10-vanilla-cnn] <=> git branch, blank means not in a git repo
bballdave025@MY-MACHINE ~/my_repos_dwb/fhtw-paper-code-prep
$ find . -type f -iname "*.py" | wc -l
18
=====..........................................................retval=0.........

(vanilla_cnn)  <=> conda environment, blank means none activated
[cifar10-vanilla-cnn] <=> git branch, blank means not in a git repo
bballdave025@MY-MACHINE ~/my_repos_dwb/fhtw-paper-code-prep
$ find . -type f -iname "*.sh" | sort | wc -l
17
=====..........................................................retval=0.........

(vanilla_cnn)  <=> conda environment, blank means none activated
[cifar10-vanilla-cnn] <=> git branch, blank means not in a git repo
bballdave025@MY-MACHINE ~/my_repos_dwb/fhtw-paper-code-prep
$ find . -type f -iname "*.sh" | sort | head
./bin/audit_backup.sh
./bin/backup_everything.sh
./bin/backup_notebook_bakstamp.sh
./bin/backup_notebooks.sh
./bin/elapsed.sh
./bin/fix_firstline_glitches.sh
./bin/start_cifar_lab.sh
./bin/sys_capture.sh
./bin/system_info_bash_fallbacks.sh
./structure.sh
=====..........................................................retval=0.........

(vanilla_cnn)  <=> conda environment, blank means none activated
[cifar10-vanilla-cnn] <=> git branch, blank means not in a git repo
bballdave025@MY-MACHINE ~/my_repos_dwb/fhtw-paper-code-prep
$ find . -type f -iname "*.sh" | sort | tr '\n' '\0' | xargs -I{} -0 bash -c 'orig={}; sixtydash="--------------------------------------------------"; tendash="----------"; echo "${sixtydash}"; echo "Script at:"; echo "$(realpath {})"; echo "${tendash}"; cat {}; echo "${sixtydash}"; echo;'
--------------------------------------------------
Script at:
/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/bin/audit_backup.sh
----------
#!/usr/bin/env bash
# Standalone audit for an existing backup tarball.
# Usage: audit_backup.sh <archive.tar.gz> <tag_path>
set -euo pipefail
OUT="${1:?Usage: $(basename "$0") <archive.tar.gz> <tag>}"
TAG="${2:?tag path e.g. test_project_bash/p_01}"

LIST="$(tar -tzf "$OUT")"
has(){ echo "$LIST" | grep -Eq "$1"; }

report(){
  local label="$1" pat="$2" warn="$3"
  if has "$pat"; then printf "  - %-32s OK\n" "$label"
  else printf "  - %-32s %s\n" "$label" "$warn"
  fi
}

echo "=== Quick audit for $(basename "$OUT") ==="
report "README_<tag>.md present?" "(^|.*/)${TAG}/README_.*\\.md$" "WARN: not found"
report "notebooks/*.ipynb present?" "(^|.*/)${TAG}/notebooks/.*\\.ipynb$" "WARN: none found"
report "scripts/*.py present?" "(^|.*/)${TAG}/scripts/.*\\.py$" "WARN: none found"
report "__init__.py at tag root?" "(^|.*/)${TAG}/__init__\\.py$" "WARN: not found"
report "scripts/__init__.py?" "(^|.*/)${TAG}/scripts/__init__\\.py$" "WARN: not found"
report "env specs included?" "(^|.*/)environment_specifications(/|$)" "WARN: not included"
report "bin/ included?" "(^|.*/)bin(/|$)" "WARN: not included"
report "firstline audit (before)?" "(^|.*/)_staging_[^/]+/backup_meta/firstline_before\\.txt$" "WARN: not included"
report "firstline audit (after)?" "(^|.*/)_staging_[^/]+/backup_meta/firstline_after\\.txt$" "INFO: not run"
report "validate_env output?" "(^|.*/)_staging_[^/]+/backup_meta/validate_env_.*\\.txt$" "INFO: validate_env not run"
report "sys_capture status?" "(^|.*/)_staging_[^/]+/backup_meta/sys_capture_status\\.txt$" "INFO: sys_capture not run"
--------------------------------------------------

--------------------------------------------------
Script at:
/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/bin/backup_everything.sh
----------
#!/usr/bin/env bash
# @file : backup_everything.sh
# Backup an experiment tag plus optional extras, with firstline audits.
# Usage:
#   backup_everything.sh <ROOT_DIR> <tag_path> [--expanded] [--skip-sys] [--skip-nbs] [--fix-firstline]
# Notes:
# - --expanded bundles: environment_specifications/, bin/, and validate_env.py output.
# - We DO NOT call backup_notebook_bakstamp.sh (that’s for one-off notebooks).
#
# Typical COMPLETE Usage example
#
# ROOT=~/my_repos_dwb/fhtw-paper-code-prep
#
# #  Tight backup with “before” firstline report
# "$ROOT/bin/backup_everything.sh" "$ROOT" test_project_bash/p01
#
# #  Expanded backup (adds env specs, bin/, validate_env.py output) + apply
# #+ fixes and include “after” report
# "$ROOT/bin/backup_everything.sh" "$ROOT" test_project_bash/p_01 --expanded --fix-firstline
#
set -euo pipefail

if (( $# < 2 )); then
  echo "Usage: $(basename "$0") <ROOT_DIR> <tag_path> [--expanded] [--skip-sys] [--skip-nbs] [--fix-firstline]" >&2
  exit 2
fi

ROOT="$1"; shift
TAG="$1";  shift

EXPANDED=0
RUN_SYS=1
RUN_NBS=1
DO_FIX_FIRSTLINE=0
while (( $# )); do
  case "$1" in
    --expanded) EXPANDED=1 ;;
    --skip-sys) RUN_SYS=0 ;;
    --skip-nbs) RUN_NBS=0 ;;
    --fix-firstline) DO_FIX_FIRSTLINE=1 ;;
    *) echo "Unknown option: $1" >&2; exit 2 ;;
  esac
  shift
done

cd "$ROOT"

TS="$(date +'%s_%Y-%m-%dT%H%M%S%z')"
BKP_DIR="$ROOT/backups"
mkdir -p "$BKP_DIR"

# Staging area for meta artifacts to include in the tarball
STAGE="$BKP_DIR/_staging_$TS"
mkdir -p "$STAGE/backup_meta"
META="$STAGE/backup_meta"

# 1) Firstline audit (before) via fix_firstline_glitches.sh --dry-run
FIXER="$ROOT/bin/fix_firstline_glitches.sh"
if [[ -x "$FIXER" ]]; then
  echo "[info] Running firstline audit (before) ..."
  if "$FIXER" --dry-run >"$META/firstline_before.txt" 2>"$META/firstline_before.err"; then
    echo "ok" > "$META/firstline_before.status"
  else
    echo "nonzero-exit" > "$META/firstline_before.status"
  fi
else
  echo "[info] No fixer found at $FIXER (skipping firstline audit)."
fi

# 2) Optional: perform fixes and capture after-report
if [[ -x "$FIXER" && $DO_FIX_FIRSTLINE -eq 1 ]]; then
  echo "[info] Applying firstline fixes (live) ..."
  if "$FIXER" >"$META/firstline_after.txt" 2>"$META/firstline_after.err"; then
    echo "ok" > "$META/firstline_after.status"
  else
    echo "nonzero-exit" > "$META/firstline_after.status"
  fi
fi

# 3) Optional helpers
if (( RUN_SYS )) && [[ -x "$ROOT/bin/sys_capture.sh" ]]; then
  echo "[info] Running sys_capture.sh ..."
  "$ROOT/bin/sys_capture.sh" >"$META/sys_capture_stdout.txt" 2>"$META/sys_capture_stderr.txt" || echo "nonzero-exit" > "$META/sys_capture_status.txt"
  [[ -f "$META/sys_capture_status.txt" ]] || echo "ok" > "$META/sys_capture_status.txt"
fi

if (( RUN_NBS )) && [[ -x "$ROOT/bin/backup_notebooks.sh" ]]; then
  echo "[info] Running backup_notebooks.sh (all notebooks) ..."
  "$ROOT/bin/backup_notebooks.sh" >"$META/backup_notebooks_stdout.txt" 2>"$META/backup_notebooks_stderr.txt" || echo "nonzero-exit" > "$META/backup_notebooks_status.txt"
  [[ -f "$META/backup_notebooks_status.txt" ]] || echo "ok" > "$META/backup_notebooks_status.txt"
fi

# 4) Validate env (always validate_env.py if present)
if (( EXPANDED )) && [[ -f "$ROOT/validate_env.py" ]]; then
  echo "[info] Running validate_env.py ..."
  { python "$ROOT/validate_env.py"; } >"$META/validate_env_stdout.txt" 2>"$META/validate_env_stderr.txt" || true
fi

# 5) Build include list
INCLUDE_PATHS=( "$TAG" "$STAGE" )
if (( EXPANDED )); then
  [[ -d environment_specifications ]] && INCLUDE_PATHS+=( environment_specifications )
  [[ -d bin ]]                        && INCLUDE_PATHS+=( bin )
fi

OUT_BASE="$(echo "$TAG" | tr '/' '_')"
OUT_NAME="${OUT_BASE}_$TS"
[[ $EXPANDED -eq 1 ]] && OUT_NAME="${OUT_BASE}_plus_env_and_bin_$TS"
OUT="$BKP_DIR/${OUT_NAME}.tar.gz"

# 6) Create archive (aligns with your atreea-style exclusions)
tar --create --gzip --file "$OUT" \
  --exclude='*/outputs/*' \
  --exclude='*/datasets/*' \
  --exclude='*/models/*' \
  --exclude='*/what_was_inside_notebook_directory_try1/*' \
  --exclude='*/.ipynb_checkpoints/*' \
  --exclude='*/__pycache__/*' \
  --exclude='*/bash_terminal_io_lab_notebooks/*' \
  --exclude='*/.DS_Store' \
  "${INCLUDE_PATHS[@]}"

# 7) Quick verify + checksum
COUNT=$(tar -tzf "$OUT" | wc -l | awk '{print $1}')
echo "[ok] Archive contains $COUNT entries."
sha256sum "$OUT" | tee "$OUT.sha256" >/dev/null
echo "[ok] Backup written: $OUT"
echo "[ok] SHA256 saved:   $OUT.sha256"

# 8) Inclusion audit (tolerant patterns)
AUDIT_LOG="$BKP_DIR/${OUT_NAME}.audit.txt"
LIST="$(tar -tzf "$OUT")"

has() { echo "$LIST" | grep -Eq "$1"; }

{
  echo "=== Inclusion audit for $OUT_NAME ==="
  echo "Timestamp: $TS"
  echo
  echo "[Expect] Tag root: $TAG"

  # Allow optional ./ prefix and be resilient
  has "(^|.*/)${TAG}/README_.*\\.md$" \
    && echo "  - README_<tag>.md present?    OK" \
    || echo "  - README_<tag>.md present?    WARN: not found"

  has "(^|.*/)${TAG}/notebooks/.*\\.ipynb$" \
    && echo "  - notebooks/*.ipynb present?  OK" \
    || echo "  - notebooks/*.ipynb present?  WARN: none found"

  has "(^|.*/)${TAG}/scripts/.*\\.py$" \
    && echo "  - scripts/*.py present?       OK" \
    || echo "  - scripts/*.py present?       WARN: none found"

  has "(^|.*/)${TAG}/__init__\\.py$" \
    && echo "  - __init__.py at tag root?    OK" \
    || echo "  - __init__.py at tag root?    WARN: not found"

  has "(^|.*/)${TAG}/scripts/__init__\\.py$" \
    && echo "  - scripts/__init__.py?        OK" \
    || echo "  - scripts/__init__.py?        WARN: not found"

  if (( EXPANDED )); then
    has "(^|.*/)environment_specifications(/|$)" \
      && echo "  - environment_specifications/? OK" \
      || echo "  - environment_specifications/? WARN: not included"

    has "(^|.*/)bin(/|$)" \
      && echo "  - bin/?                       OK" \
      || echo "  - bin/?                       WARN: not included"

    # Staging meta (handle absolute-to-relative path strip by tar)
    has "(^|.*/)_staging_[^/]+/backup_meta/validate_env_.*\\.txt$" \
      && echo "  - validate_env output in backup_meta/? OK" \
      || echo "  - validate_env output in backup_meta/? INFO: validate_env not run or produced no files"
  fi

  has "(^|.*/)_staging_[^/]+/backup_meta/firstline_before\\.txt$" \
    && echo "  - firstline audit (before)?   OK" \
    || echo "  - firstline audit (before)?   WARN: not included"

  has "(^|.*/)_staging_[^/]+/backup_meta/firstline_after\\.txt$" \
    && echo "  - firstline audit (after)?    OK" \
    || echo "  - firstline audit (after)?    INFO: not run"

  has "(^|.*/)_staging_[^/]+/backup_meta/sys_capture_status\\.txt$" \
    && echo "  - sys_capture status?         OK" \
    || echo "  - sys_capture status?         INFO: sys_capture not run"

  echo
} > "$AUDIT_LOG"
echo "[ok] Wrote audit: $AUDIT_LOG"

# 9) Clean up staging (the tarball already contains it)
rm -rf "$STAGE"
--------------------------------------------------

--------------------------------------------------
Script at:
/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/bin/backup_notebook_bakstamp.sh
----------
#!/usr/bin/env bash
# backup_notebook_bakstamp.sh
# Create timestamped .bak copies of one or more notebooks in-place.
# Usage: backup_notebook_bakstamp.sh NOTEBOOK.ipynb [MORE.ipynb ...]
# Pattern: file.ipynb.$(date +'%s_%Y-%m-%dT%H%M%S%z').bak

set -Eeuo pipefail

if [[ $# -lt 1 ]]; then
  echo "Usage: $(basename "$0") NOTEBOOK.ipynb [MORE.ipynb ...]" >&2
  exit 2
fi

stamp="$(date +'%s_%Y-%m-%dT%H%M%S%z')"

for nb in "$@"; do
  if [[ ! -f "$nb" ]]; then
    echo "Skipping non-existent: $nb" >&2
    continue
  fi
  dir="$(dirname "$nb")"
  base="$(basename "$nb")"
  bak="${dir}/${base}.${stamp}.bak"
  cp -v -- "$nb" "$bak"
done
--------------------------------------------------

--------------------------------------------------
Script at:
/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/bin/backup_notebooks.sh
----------
\
#!/usr/bin/env bash
# backup_notebooks.sh
# Make timestamped .bak copies of notebooks in ./notebooks/ (or a chosen dir).
# Optionally prune older backups, keeping only the most recent N per notebook.
#
# Timestamp pattern: $(date +'%s_%Y-%m-%dT%H%M%S%z')

set -Eeuo pipefail

#PROJECT_DIR="${PROJECT_DIR:-$PWD}"
PROJECT_DIR=${PROJECT_DIR:-$HOME/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_01}
NB_DIR_DEFAULT="notebooks"
NB_DIR=""
PRUNE=0
KEEP="${KEEP:-3}"

usage() {
  cat <<USAGE
Usage: $(basename "$0") [options] [NOTEBOOK.ipynb ...]

Without arguments, operates on all *.ipynb under ./notebooks/.

Options:
  -p, --project DIR     Project directory (default: \$PWD)
  -n, --nbdir DIR       Notebooks subdirectory relative to project (default: notebooks)
  --prune               After backup, prune old backups, keeping last N (see --keep)
  -k, --keep N          Number of backups to keep per notebook when pruning (default: 3)
  -h, --help            Show this help and exit

Env overrides:
  PROJECT_DIR, KEEP

Examples:
  $(basename "$0")
  $(basename "$0") --prune --keep 5
  $(basename "$0") -p "$HOME/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_01" --prune
  $(basename "$0") notebooks/01_model_build_p_01.ipynb
USAGE
}

# Parse flags
args=()
while [[ $# -gt 0 ]]; do
  case "$1" in
    -p|--project) PROJECT_DIR="$2"; shift 2;;
    -n|--nbdir) NB_DIR="$2"; shift 2;;
    --prune) PRUNE=1; shift;;
    -k|--keep) KEEP="$2"; shift 2;;
    -h|--help) usage; exit 0;;
    --) shift; break;;
    -*)
      echo "Unknown option: $1" >&2
      usage; exit 2
      ;;
    *)
      args+=("$1"); shift;;
  esac
done
set -- "${args[@]}"

# Resolve notebook directory
if [[ -z "${NB_DIR}" ]]; then
  NB_DIR="${PROJECT_DIR}/${NB_DIR_DEFAULT}"
elif [[ "${NB_DIR}" != /* ]]; then
  NB_DIR="${PROJECT_DIR}/${NB_DIR}"
fi

if [[ ! -d "$NB_DIR" ]]; then
  echo "No notebooks directory found at: $NB_DIR" >&2
  exit 1
fi

stamp="$(date +'%s_%Y-%m-%dT%H%M%S%z')"

backup_one() {
  local nb="$1"
  if [[ ! -f "$nb" ]]; then
    echo "Skipping non-existent: $nb" >&2
    return 0
  fi
  local bak="${nb}.${stamp}.bak"
  cp -v -- "$nb" "$bak"
  if [[ $PRUNE -eq 1 ]]; then
    prune_backups "$nb"
  fi
}

prune_backups() {
  local nb="$1"
  # List backups for this notebook, sorted by mtime descending, keep first $KEEP, remove rest
  # Note: relies on filenames without newlines; standard in practice.
  local dir base
  dir="$(dirname "$nb")"
  base="$(basename "$nb")"
  # Gather list
  mapfile -t all_baks < <(ls -1t -- "$dir/${base}."*".bak" 2>/dev/null || true)
  if (( ${#all_baks[@]} > KEEP )); then
    # Determine deletions
    local to_delete=( "${all_baks[@]:KEEP}" )
    for old in "${to_delete[@]}"; do
      echo "Pruning old backup: $old"
      rm -f -- "$old"
    done
  fi
}

# Collect notebooks to operate on
nb_list=()
if [[ $# -gt 0 ]]; then
  # Use provided paths
  for p in "$@"; do
    if [[ -d "$p" ]]; then
      while IFS= read -r -d '' f; do nb_list+=("$f"); done < <(find "$p" -maxdepth 1 -type f -name "*.ipynb" -print0)
    else
      nb_list+=("$p")
    fi
  done
else
  while IFS= read -r -d '' f; do nb_list+=("$f"); done < <(find "$NB_DIR" -maxdepth 1 -type f -name "*.ipynb" -print0)
fi

if [[ ${#nb_list[@]} -eq 0 ]]; then
  echo "No notebooks found to back up." >&2
  exit 0
fi

for nb in "${nb_list[@]}"; do
  backup_one "$nb"
done
--------------------------------------------------

--------------------------------------------------
Script at:
/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/bin/elapsed.sh
----------
elapsed() {
  local state_file="/tmp/.elapsed_timer"

  case "$1" in
    -s|--start)
      date +%s > "$state_file"
      echo "Stopwatch started."
      ;;
    -p|--stop)
      if [[ -f "$state_file" ]]; then
        local start=$(<"$state_file")
        local now=$(date +%s)
        local diff=$((now - start))
        echo "$diff"
        rm -f "$state_file"
      else
        echo "No stopwatch running. Use --start to begin."
      fi
      ;;
    -c|--check)
      if [[ -f "$state_file" ]]; then
        local start=$(<"$state_file")
        local now=$(date +%s)
        local diff=$((now - start))
        echo "$diff"
      else
        echo "No stopwatch running. Use --start to begin."
      fi
      ;;
    -h|--help|*)
      cat <<EOF
Usage: elapsed [OPTION]

Options:
  -s, --start   Start the stopwatch
  -c, --check   Show elapsed time (in seconds, stopwatch keeps running)
  -p, --stop    Stop the stopwatch and show elapsed time (in seconds)
  -h, --help    Show this help message

If run without an option, this help will be displayed.
EOF
      ;;
  esac
}
--------------------------------------------------

--------------------------------------------------
Script at:
/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/bin/fix_firstline_glitches.sh
----------
#!/usr/bin/env bash
# Fix common first-line glitches in *.sh and *.py:
#  - Remove a standalone backslash "\" on line 1
#  - Remove a UTF-8 BOM on line 1
# Optional: --add-shebang will add a shebang to files that lack one,
#           choosing based on extension (.sh -> bash, .py -> python3).
#
# It first creates a repo backup tarball (no spaces in name), then
# prints a per-file report of changes.
#
# Usage:
#   fix_firstline_glitches.sh [--root <dir>] [--dry-run] [--add-shebang]
#                              [--ext ".sh,.py"] [--no-backup]
#
# Defaults:
#   --root: git root if available, else CWD
#   --ext:  ".sh,.py"
#   backup: ON (writes to <root>/backups)
#
#
# More-complete Usage examples
#
# # Normal run at repo root,default extensions (.sh,.py),with backup and report
# bin/fix_firstline_glitches.sh
#
# # Dry-run (no changes), just show what *would* be done
# bin/fix_firstline_glitches.sh --dry-run
#
# # Opt-in: also add shebangs to files that lack one (based on extension)
# bin/fix_firstline_glitches.sh --add-shebang
# 
# # Run on a subdirectory only (e.g., your tag dir)
# bin/fix_firstline_glitches.sh --root "$ROOT/test_project_bash/p_01"
#
# # Different extensions (add .ps1 if desired)
# bin/fix_firstline_glitches.sh --ext ".sh,.py,.ps1"
#


set -euo pipefail

# --- args ---
ROOT=""
DRY_RUN=0
ADD_SHEBANG=0
DO_BACKUP=1
EXT_LIST=".sh,.py"

while (( $# )); do
  case "$1" in
    --root) ROOT="${2:?}"; shift 2;;
    --dry-run) DRY_RUN=1; shift;;
    --add-shebang) ADD_SHEBANG=1; shift;;
    --ext) EXT_LIST="${2:?}"; shift 2;;
    --no-backup) DO_BACKUP=0; shift;;
    -h|--help)
      cat <<EOH
Usage: $(basename "$0") [--root <dir>] [--dry-run] [--add-shebang] [--ext ".sh,.py"] [--no-backup]
EOH
      exit 0;;
    *)
      echo "Unknown option: $1" >&2; exit 2;;
  esac
done

# --- resolve root ---
if [[ -z "$ROOT" ]]; then
  if git rev-parse --show-toplevel >/dev/null 2>&1; then
    ROOT=$(git rev-parse --show-toplevel)
  else
    ROOT=$(pwd -P)
  fi
fi
cd "$ROOT"

# --- sed in-place portability ---
if sed --version >/dev/null 2>&1; then
  SED_INPLACE=(sed -i)
else
  SED_INPLACE=(sed -i '')
fi

timestamp() { date +'%s_%Y-%m-%dT%H%M%S%z'; }

# --- backup ---
BACKUP_PATH=""
if (( DO_BACKUP )); then
  TS="$(timestamp)"
  mkdir -p "$ROOT/backups"
  BACKUP_BASENAME="fix_firstline_glitches_${TS}.tar.gz"
  BACKUP_PATH="$ROOT/backups/$BACKUP_BASENAME"
  echo "Backing up to $BACKUP_PATH"
  # Exclude the backups dir (and this file), and suppress 'file changed as we read it' exits
  tar --create --gzip --file "$BACKUP_PATH" \
      --exclude='./backups/*' \
      --exclude="./backups/$BACKUP_BASENAME" \
      --exclude='*/outputs/*' \
      --exclude='*/datasets/*' \
      --exclude='*/models/*' \
      --exclude='*/what_was_inside_notebook_directory_try1/*' \
      --exclude='*/.ipynb_checkpoints/*' \
      --exclude='*/__pycache__/*' \
      --exclude='*/bash_terminal_io_lab_notebooks/*' \
      --exclude='*/.DS_Store' \
      --warning=no-file-changed \
      --ignore-failed-read \
      . || true
  sha256sum "$BACKUP_PATH" | tee "$BACKUP_PATH.sha256" >/dev/null || true
  echo "Done backing up"
  echo
fi

# --- collect target files ---
# Turn ".sh,.py" into a find predicate
IFS=',' read -r -a exts <<< "$EXT_LIST"
pred=()
for e in "${exts[@]}"; do
  e="${e#"."}"           # strip leading dot if present
  pred+=( -iname "*.${e}" -o )
done
# drop trailing -o
unset 'pred[${#pred[@]}-1]'

mapfile -d '' FILES < <(find . -type f \( "${pred[@]}" \) -print0)

# --- helpers ---
strip_firstline_backslash() {
  # remove a line-1 that is exactly "\" (standalone backslash)
  "${SED_INPLACE[@]}" '1{/^\\$/d;}' "$1"
}

strip_bom() {
  # remove UTF-8 BOM at start of file
  if sed --version >/dev/null 2>&1; then
    "${SED_INPLACE[@]}" '1s/^\xEF\xBB\xBF//g' "$1"
  else
    "${SED_INPLACE[@]}" $'1s/^\xEF\xBB\xBF//' "$1"
  fi
}

file_is_text() {
  LC_ALL=C grep -Iq . -- "$1"
}

first2_bytes_hex() {
  head -c 2 -- "$1" | xxd -p -l 2 2>/dev/null || true
}

has_shebang() {
  [[ "$(head -c 2 -- "$1" 2>/dev/null || true)" == '#!' ]]
}

add_shebang_if_requested() {
  local f="$1"
  (( ADD_SHEBANG )) || return 0
  has_shebang "$f" && return 0

  # choose interpreter by extension
  case "${f##*.}" in
    sh)  line='#!/usr/bin/env bash';;
    py)  line='#!/usr/bin/env python3';;
    *)   return 0;;
  esac

  # Prepend shebang safely (preserving existing content)
  if (( DRY_RUN )); then
    return 0
  fi
  # If file starts with BOM, remove it first so shebang is byte 0
  strip_bom "$f"
  tmp="$(mktemp)"
  printf '%s\n' "$line" > "$tmp"
  cat -- "$f" >> "$tmp"
  mv -- "$tmp" "$f"
  chmod +x "$f" || true
  return 0
}

# --- processing ---
echo "Checking and changing files"
CHANGES=0
for f in "${FILES[@]}"; do
  # skip binaries just in case
  file_is_text "$f" || { echo "$f : skipped (binary)"; continue; }

  pre_hex="$(first2_bytes_hex "$f")"
  pre_hassb=0; has_shebang "$f" && pre_hassb=1

  action_msgs=()
  changed=0

  # simulate or apply
  if (( DRY_RUN )); then
    # dry-run: check whether we'd change anything
    # Check BOM:
    if [[ "$pre_hex" =~ ^ef ]]; then
      action_msgs+=( "UTF-8 BOM stripped (would)" )
    fi
    # Check standalone backslash on L1
    if head -n1 -- "$f" | grep -qE '^[\\]$'; then
      action_msgs+=( "first-line backslash stripped (would)" )
    fi
    # Shebang add?
    if (( ! pre_hassb )); then
      if (( ADD_SHEBANG )); then
        case "${f##*.}" in
          sh|py) action_msgs+=( "shebang added (would)" );;
        esac
      else
        action_msgs+=( "doesn't start with shebang" )
      fi
    fi
  else
    # live edits
    # strip BOM if present
    if [[ "$pre_hex" =~ ^ef ]]; then
      strip_bom "$f"; action_msgs+=( "UTF-8 BOM stripped" ); changed=1
    fi
    # strip standalone backslash on line 1
    if head -n1 -- "$f" | grep -qE '^[\\]$'; then
      strip_firstline_backslash "$f"; action_msgs+=( "first-line backslash stripped" ); changed=1
    fi
    # optionally add shebang
    if (( ! pre_hassb )); then
      if add_shebang_if_requested "$f"; then
        if (( ADD_SHEBANG )); then
          action_msgs+=( "shebang added" ); changed=1
        else
          action_msgs+=( "doesn't start with shebang" )
        fi
      fi
    fi
  fi

  if (( changed )); then
    (( CHANGES++ ))
    echo "$f : ${action_msgs[*]}"
  else
    # print consistent status line
    if ((${#action_msgs[@]})); then
      echo "$f : ${action_msgs[*]}"
    else
      echo "$f : no change"
    fi
  fi
done

echo
if (( DRY_RUN )); then
  echo "Dry-run complete. Files that say '(would)' would be modified."
else
  echo "Edits complete. $CHANGES file(s) modified."
fi

# Exit nonzero in dry-run if we WOULD change anything (useful in CI)
if (( DRY_RUN && CHANGES > 0 )); then
  exit 3
fi

exit 0
--------------------------------------------------

--------------------------------------------------
Script at:
/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/bin/start_cifar_lab.sh
----------
#!/usr/bin/env bash
# start_cifar_lab.sh
# Helper to cd into your CIFAR-10 project tag directory, activate env, and launch JupyterLab.

set -Eeuo pipefail

PROJECT_DIR="${PROJECT_DIR:-$HOME/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_01}"
ENV_NAME="${ENV_NAME:-vanillacnn}"
NO_BROWSER="${NO_BROWSER:-0}"
PORT="${PORT:-8888}"
ENSURE_KERNEL="${ENSURE_KERNEL:-0}"
KERNEL_NAME="${KERNEL_NAME:-vanillacnn}"
KERNEL_DISPLAY="${KERNEL_DISPLAY:-Python (vanillacnn)}"

usage() {
  cat <<USAGE
Usage: $(basename "$0") [options]

Options:
  -p, --project DIR      Project tag directory (default: \$HOME/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_01)
  -e, --env NAME         Conda environment name (default: vanillacnn)
  -k, --ensure-kernel    Ensure ipykernel is registered for this env (default: off)
  -b, --no-browser       Launch JupyterLab without opening a browser (default: off)
  -P, --port PORT        Jupyter port (default: 8888)
  -h, --help             Show this help and exit
USAGE
}

while [[ $# -gt 0 ]]; do
  case "$1" in
    -p|--project) PROJECT_DIR="$2"; shift 2;;
    -e|--env) ENV_NAME="$2"; shift 2;;
    -k|--ensure-kernel) ENSURE_KERNEL="1"; shift;;
    -b|--no-browser) NO_BROWSER="1"; shift;;
    -P|--port) PORT="$2"; shift 2;;
    -h|--help) usage; exit 0;;
    *) echo "Unknown option: $1" >&2; usage; exit 2;;
  esac
done

# Init conda or micromamba
if command -v conda >/dev/null 2>&1; then
  eval "$(conda shell.bash hook)"
  if ! conda activate "$ENV_NAME" 2>/dev/null; then
    echo "Conda env '$ENV_NAME' not found." >&2
    echo "Create it, e.g." >&2
    echo "  conda create -n $ENV_NAME --file $HOME/my_repos_dwb/fhtw-paper-code-prep/environment_specifications/conda-linux-64.lock" >&2
    echo "OR possibly" >&2
    echo "  conda create -n $ENV_NAME --file conda-linux-64.lock" >&2
    exit 1
  fi
elif command -v micromamba >/dev/null 2>&1; then
  eval "$(micromamba shell hook -s bash)"
  if ! micromamba activate "$ENV_NAME" 2>/dev/null; then
    echo "Micromamba env '$ENV_NAME' not found." >&2
    echo "Create it, e.g." >&2
    echo "   micromamba create -n $ENV_NAME --file $HOME/my_repos_dwb/fhtw-paper-code-prep/environment_specifications/conda-linux-64.lock" >&2
    echo "OR possibly" >&2
    echo "   micromamba create -n $ENV_NAME --file conda-linux-64.lock" >&2
    exit 1
  fi
else
  echo "Error: Neither conda nor micromamba found on PATH." >&2
  exit 1
fi

# Optional: ensure kernel
if [[ "${ENSURE_KERNEL}" == "1" ]]; then
  python - <<PY
import subprocess, sys
try:
    import jupyter_client  # noqa
except Exception:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ipykernel"])
subprocess.check_call([sys.executable, "-m", "ipykernel", "install", "--user",
                       "--name", "${KERNEL_NAME}", "--display-name", "${KERNEL_DISPLAY}"])
print("Kernel ensured: ${KERNEL_DISPLAY}")
PY
fi

cd "$PROJECT_DIR"

if command -v jupyter-lab >/dev/null 2>&1; then
  if [[ "$NO_BROWSER" == "1" ]]; then
    exec jupyter-lab --no-browser --port "$PORT"
  else
    exec jupyter-lab --port "$PORT"
  fi
else
  if [[ "$NO_BROWSER" == "1" ]]; then
    exec jupyter lab --no-browser --port "$PORT"
  else
    exec jupyter lab --port "$PORT"
  fi
fi
--------------------------------------------------

--------------------------------------------------
Script at:
/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/bin/sys_capture.sh
----------
#!/usr/bin/env bash
# sys_capture.sh
# Capture env + system + git state for a run.
#
# Defaults target a project-level logs dir:
#   $PROJECT_ROOT/logs/system_<stamp>/
# You can override to a tag-specific location with --tag p_01 (then $PROJECT_ROOT/test_project_bash/p_01/logs/...)
#
# Usage:
#   sys_capture.sh [--project ROOT] [--tag TAGDIR_NAME] [--out-root DIR] [--note TEXT]
#                  [--no-git] [--no-conda-lock] [--no-pip-freeze] [--json-only]
#
# Examples:
#   sys_capture.sh --project "$HOME/my_repos_dwb/fhtw-paper-code-prep"
#   sys_capture.sh --project "$PWD" --tag p_01 --note "seed=137 try1"
#   RUN_ID="p_01_try_$(date +'%Y-%m-%dT%H%M%S%z')" sys_capture.sh --project "$PWD" --tag p_01
#
set -Eeuo pipefail

PROJECT_ROOT="${PROJECT_ROOT:-}"
TAG_NAME=""
OUT_ROOT=""
NOTE=""
DO_GIT=1
DO_CONDA_LOCK=1
DO_PIP_FREEZE=1
JSON_ONLY=0

usage() {
  sed -n '1,40p' "$0" | sed 's/^# \{0,1\}//'
  exit 0
}

# --- parse args ---
while [[ $# -gt 0 ]]; do
  case "$1" in
    -p|--project) PROJECT_ROOT="$2"; shift 2;;
    -t|--tag) TAG_NAME="$2"; shift 2;;
    -o|--out-root) OUT_ROOT="$2"; shift 2;;
    -n|--note) NOTE="$2"; shift 2;;
    --no-git) DO_GIT=0; shift;;
    --no-conda-lock) DO_CONDA_LOCK=0; shift;;
    --no-pip-freeze) DO_PIP_FREEZE=0; shift;;
    --json-only) JSON_ONLY=1; shift;;
    -h|--help) usage;;
    *) echo "Unknown option: $1" >&2; usage;;
  esac
done

# --- resolve project root ---
if [[ -z "${PROJECT_ROOT}" ]]; then
  # auto-detect by looking for repo markers
  if [[ -f "./README.md" && -d "./test_project_bash" ]]; then
    PROJECT_ROOT="$(pwd)"
  elif [[ -d "../test_project_bash" ]]; then
    PROJECT_ROOT="$(cd .. && pwd)"
  else
    PROJECT_ROOT="$(pwd)"  # fallback
  fi
fi

# --- compute base output dir ---
STAMP="${RUN_ID:-$(date +'%s_%Y-%m-%dT%H%M%S%z')}"
if [[ -n "${OUT_ROOT}" ]]; then
  BASE_OUT="${OUT_ROOT}"
elif [[ -n "${TAG_NAME}" ]]; then
  BASE_OUT="${PROJECT_ROOT}/test_project_bash/${TAG_NAME}/logs"
else
  BASE_OUT="${PROJECT_ROOT}/logs"
fi
OUT="${BASE_OUT}/system_${STAMP}"
mkdir -p -- "$OUT"

echo "→ Capturing to: $OUT"

# --- 1) Git snapshot ---
if (( DO_GIT )); then
  if command -v git >/dev/null 2>&1; then
    git -C "$PROJECT_ROOT" rev-parse HEAD > "$OUT/git_commit.txt" 2>/dev/null || true
    git -C "$PROJECT_ROOT" status --porcelain=v1 > "$OUT/git_status.txt" 2>/dev/null || true
    git -C "$PROJECT_ROOT" diff > "$OUT/git_diff.patch" 2>/dev/null || true
  fi
fi

# --- 2) Conda/Python/env ---
if command -v conda >/dev/null 2>&1; then
  conda info > "$OUT/conda_info.txt" 2>/dev/null || true
  conda env export --from-history > "$OUT/environment.yml" 2>/dev/null || true
  if (( DO_CONDA_LOCK )); then
    conda list --explicit > "$OUT/conda-linux-64.lock" 2>/dev/null || true
  fi
fi
python -V > "$OUT/python_version.txt" 2>/dev/null || true
if (( DO_PIP_FREEZE )); then
  python -m pip freeze > "$OUT/requirements.txt" 2>/dev/null || true
fi
jupyter kernelspec list > "$OUT/kernelspecs.txt" 2>/dev/null || true

# --- 3) System snapshot ---
uname -a > "$OUT/uname.txt" 2>/dev/null || true
( command -v lsb_release >/dev/null && lsb_release -a ) > "$OUT/lsb_release.txt" 2>/dev/null || true
( command -v lscpu >/dev/null && lscpu ) > "$OUT/lscpu.txt" 2>/dev/null || true
( command -v free >/dev/null && free -h ) > "$OUT/mem.txt" 2>/dev/null || true
( command -v df  >/dev/null && df -hP . ) > "$OUT/disk.txt" 2>/dev/null || true

# --- 4) GPU snapshot (non-fatal) ---
( command -v nvidia-smi >/dev/null && nvidia-smi -q ) > "$OUT/nvidia_smi.txt" 2>/dev/null || true

# --- 5) Structured JSON summary (Python) ---
python - <<'PY' > "$OUT/system_info.json" 2>/dev/null || true
import json, sys, platform, os, subprocess
def cmd(args):
    try: return subprocess.check_output(args, text=True).strip()
    except Exception: return None
info = {
  "python": cmd([sys.executable, "--version"]),
  "platform": platform.platform(),
  "uname": dict(zip(("system","node","release","version","machine","processor"),
                    platform.uname())),
  "conda_prefix": os.environ.get("CONDA_PREFIX"),
  "git_commit": cmd(["git","rev-parse","HEAD"]),
}
try:
    import tensorflow as tf; info["tensorflow"] = tf.__version__
except Exception: pass
try:
    import numpy as np; info["numpy"] = np.__version__
except Exception: pass
print(json.dumps(info, indent=2))
PY

# --- 6) Optional human note ---
if [[ -n "${NOTE}" ]]; then
  printf "%s\n" "$NOTE" > "$OUT/note.txt"
fi

# --- 7) Minimal mode: keep only JSON if requested ---
if (( JSON_ONLY )); then
  find "$OUT" -type f ! -name 'system_info.json' -delete
fi

echo "✓ Capture complete: $OUT"
--------------------------------------------------

--------------------------------------------------
Script at:
/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/bin/system_info_bash_fallbacks.sh
----------
# Env / package managers
conda info > conda_info.txt
conda list > conda_list.txt
conda list --explicit > conda-linux-64.lock  # exact builds
command -v pip >/dev/null && pip freeze > requirements.txt

# OS + kernel
uname -a > uname.txt
( command -v lsb_release >/dev/null && lsb_release -a ) > lsb_release.txt 2>/dev/null || \
  cat /etc/os-release > os_release.txt

# CPU / memory / disk
lscpu > lscpu.txt
nproc > nproc.txt
free -h > free_h.txt
df -h . > df_h_here.txt

# Python / Jupyter
which python > which_python.txt
python -V > python_version.txt
command -v jupyter >/dev/null && jupyter --version > jupyter_version.txt
command -v jupyter >/dev/null && jupyter kernelspec list > kernelspecs.txt

# Locale (helps with encoding quirks)
locale > locale.txt

# TF / Keras relevant env knobs (if any are set)
( printenv | grep -E '^(KERAS_HOME|TF_CPP_MIN_LOG_LEVEL|OMP_NUM_THREADS|TF_NUM_(INTRA|INTER)OP_THREADS)=' ) \
  > tf_env_knobs.txt

# GPU (only if present)
command -v nvidia-smi >/dev/null && nvidia-smi -L > nvidia_gpus.txt || true
--------------------------------------------------

--------------------------------------------------
Script at:
/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/structure.sh
----------
#!/usr/bin/env bash
# Usage:                 ./structure.sh <ROOT_DIR> <tag1> [tag2 ...]
#   OR
#        WITH_NB_STUBS=1 ./structure.sh <ROOT_DIR> <tag1> [tag2 ...]
#
# Creates the structure shown in your example, including:
# - README_<tag>.md
# - notebooks/*_<tag>.ipynb
#   - (Note that these are created idempotently, i.e. the are created
#      only if they do not already exist. If  WITH_NB_STUBS=1  is
#      included, minimal IPYNB files, i.e. those with enough JSON
#      content to be recognized as IPYNB files, will be created,
#      once again idempotently)
# - scripts/{py_build_model,py_train_model,py_inference,py_utils}_<tag>.py
# - scripts/{build_model,train_model,inference}_<tag>.cmd
# - scripts/py_touch.py        # untagged (per tag directory)
# - datasets/, models/, logs/, visualizations/, 
#   outputs/{csv_logs,gradcam_images}
#
#
#
#
# As far as the backup subcommand, some examples follow
#
# # Back up a single tag (tight)
# ./structure.sh backup "$HOME/my_repos_dwb/fhtw-paper-code-prep" test_project_bash/p_01
#
# # Expanded + apply firstline fixes (after audit)
# ./structure.sh backup "$HOME/my_repos_dwb/fhtw-paper-code-prep" test_project_bash/p_01 --expanded --fix-firstline
#
# # Multiple tags, expanded, no system snapshot
# ./structure.sh backup "$HOME/my_repos_dwb/fhtw-paper-code-prep" test_project_bash/p_01 test_project_bash/p_03_e2e --expanded --skip-sys
#
#


set -euo pipefail

# ---- usage/help (robust, multi-line) ----
usage() {
  cat <<EOF
Usage:
  $0 <ROOT_DIR> <tag1> [tag2 ...]

Optionally:
  WITH_NB_STUBS=1 $0 <ROOT_DIR> <tag1> [tag2 ...]

Creates the scaffold; if WITH_NB_STUBS=1, also creates minimal .ipynb stubs.
Idempotent (won’t clobber existing files).

Subcommands:
  $0 backup <ROOT_DIR> <tag1> [tag2 ...] [--expanded] [--fix-firstline] [--skip-sys] [--skip-nbs]
    Runs bin/backup_everything.sh for each tag. Leaves scaffolding untouched.
EOF
}

# ---- backup subcommand passthrough ----
do_backup_subcommand() {
  # Usage: structure.sh backup <ROOT_DIR> <tag1> [tag2 ...] [flags...]
  # Flags are forwarded to bin/backup_everything.sh
  local _args=("$@")
  if (( ${#_args[@]} < 2 )); then
    echo "Usage: $0 backup <ROOT_DIR> <tag1> [tag2 ...] [--expanded] [--fix-firstline] [--skip-sys] [--skip-nbs]" >&2
    exit 2
  fi

  local _root="${_args[0]}"; shift || true
  # collect tags until a flag (starts with --)
  local _tags=()
  while (( $# )) && [[ "${1}" != --* ]]; do
    _tags+=("$1"); shift
  done
  local _flags=("$@")

  # Ensure backup script exists
  local _bkpsh="$_root/bin/backup_everything.sh"
  if [[ ! -x "$_bkpsh" ]]; then
    echo "[ERR] Not found or not executable: $_bkpsh" >&2
    echo "      Please place backup_everything.sh in \$_ROOT/bin and chmod +x." >&2
    exit 1
  fi

  # Run for each tag
  for t in "${_tags[@]}"; do
    echo "==> Backing up tag: $t"
    "$_bkpsh" "$_root" "$t" "${_flags[@]}"
    echo
  done
  exit 0
}

#old#ROOT_DIR="${1:?Usage:"\
#old#"                                  $0 <ROOT_DIR> <tag1> [tag2 ...]\n"\
#old#" (optionally) run WITH_NB_STUBS=1 $0 <ROOT_DIR> <tag1> [tag2 ...]\n"\
#old#" The second creates minimal IPYNB files. Both create IPYNB idempotently i.e.\n"\
#old#" only if they do not exist.}"

if (( $# < 2 )); then usage >&2; exit 1; fi

# Subcommand: backup
if [[ "${1:-}" == "backup" ]]; then
  shift
  do_backup_subcommand "$@"
fi

# ---- args/derived ----
ROOT_DIR=$1; shift
if [ $# -lt 1 ]; then
  echo "Need at least one tag (e.g., p_01 p_02)" >&2
  exit 2
fi
TAGS=("$@")

#  Untagged/common base (defaults to ROOT_DIR, can be overridden by env
: "${UNTAGGED_COMMON:="$ROOT_DIR"}"

# Common per-tag untagged helper
UNTAGGED_COMMON_FILES=(
  "scripts/py_touch.py"
  "scripts/normalize_eol.py"
  #".gitattributes"
)

#  Put EOL entries in for platform-specific files,
#+  thus ensuring platform independence
ensure_gitattributes_entries() {
  local base="${1:-$ROOT_DIR}"
  local repo_root
  repo_root=$(git -C "$base" rev-parse --show-toplevel 2>/dev/null || echo "$base")

  local f="$repo_root/.gitattributes"
  local -a lines=(
    "*.sh              text eol=lf"
    "*.ps1             text eol=crlf"
    "*.cmd             text eol=crlf"
    "*.py              text eol=lf"
    "*.md              text eol=lf"
    "*.ipynb           text eol=lf"
    ".gitattributes    text eol=lf"
  )

  mkdir -p "$repo_root"
  touch "$f"
  chmod 0644 "$f" 2>/dev/null || true
  # ensure file ends with newline
  tail -c1 "$f" 2>/dev/null | grep -q $'\n' || echo >> "$f"

  for line in "${lines[@]}"; do
    grep -Fqx -- "$line" "$f" || echo "$line" >> "$f"
  done
  echo "[OK] ensured .gitattributes entries at $f"
}

mkd() { mkdir -p -- "$1"; }
touch_safe() {
  local path="$1"
  mkd "$(dirname -- "$path")"
  [ -f "$path" ] || : > "$path"
}

#@TODO : Use getopt to pass in the WITH_NB_STUBS option as well as other opts
#
#  Optional: export WITH_NB_STUBS=1 to write minimal *.ipynb files
WITH_NB_STUBS="${WITH_NB_STUBS:-0}"
emit_ipynb() {
  local path="$1"
  python - <<'PY' "$path"
import json, sys, os
nb = {
  "cells":[{"cell_type":"code","execution_count":None,"metadata":{},"outputs":[],
    "source":[
      "import os, random, numpy as np, tensorflow as tf\n",
      "os.environ[\"PYTHONHASHSEED\"]=\"137\";\n",
      "random.seed(137);\n",
      "np.random.seed(137);\n",
      "tf.random.set_seed(137)\n"
    ]}],
  "metadata":{
    "kernelspec":{"display_name":"Python 3 (ipykernel)","language":"python","name":"python3"},
    "language_info":{"name":"python","version":"3.10"}
  },
  "nbformat":4,"nbformat_minor":5
}
open(sys.argv[1], "w", encoding="utf-8").write(json.dumps(nb, ensure_ascii=False, indent=1))
PY
}

## Added 1756960018_2025-09-03T222658-0600, just barely
if [ ! -d "$ROOT_DIR" ]; then
  mkd "$ROOT_DIR"
fi

ensure_gitattributes_entries "$ROOT_DIR"

echo "  have this in here."
echo "  (Note that any files that should be excluded/ignored"
echo "   are covered in the Git repo's root .gitignore file.)"
echo

# Per-tag file stems (relative to the tag root)
README_STEM="README.md"

#  Per-tag enabling import of tag-dir  and its script dir as packages
#+ tag is NOT part of filename
PACKAGE_FILES=(
  "__init__.py"
  "scripts/__init__.py"
)

# Notebooks (placed under notebooks/, with tag suffix)
NB_FILES=(
  "notebooks/00_data_exploration.ipynb"
  "notebooks/01_model_build.ipynb"
  "notebooks/02_training.ipynb"
  "notebooks/03_inference_quick_explore.ipynb"
)

#  Scripts (tag-suffixed python placeholders)
#+ These could perform the same thing as the
#+ *.sh versions and the *.ps1 versions. 
#+ We'll probably only get to one of them
#+ before the Fragmentology publication.
PY_FILES=(
  "scripts/py_build_model.py"
  "scripts/py_train_model.py"
  "scripts/py_inference.py"
  "scripts/py_utils.py"
)

#  Scripts (tag-suffixed SH placeholders)
#+ These could perform the same thing as the
#+ py_*.py versions and the *.ps1 versions. 
#+ We'll probably only get to one of them
#+ before the Fragmentology publication.
THE_SH_FILES=(
  "scripts/build_model.sh"
  "scripts/train_model.sh"
  "scripts/inference.sh"
  "scripts/py_utils.sh"
)

#  Scripts (tag-suffixed SH placeholders)
#+ These could perform the same thing as the
#+ py_*.py versions and the *.ps1 versions. 
#+ We'll probably only get to one of them
#+ before the Fragmentology publication.
THE_PS_FILES=(
  "scripts/build_model.ps1"
  "scripts/train_model.ps1"
  "scripts/inference.ps1"
)

#  Scripts (tag-suffixed CMD placeholders)
#+ We probably won't ever end up touching
#+ these, but they're here for completeness.
CMD_FILES=(
  "scripts/build_model.cmd"
  "scripts/train_model.cmd"
  "scripts/inference.cmd"
)

# Fixed directory set under each tag
DIRS=(
  "notebooks"
  "datasets"
  "models"
  "logs"
  "visualizations"
  "scripts"
  "outputs/csv_logs"
  "outputs/gradcam_images"
)

for tag in "$@"; do
  TAG_DIR="${ROOT_DIR%/}/$tag"

  # Directories
  for d in "${DIRS[@]}"; do
    mkd "$TAG_DIR/$d"
  done

  # README_<tag>.md at tag root
  touch_safe "$TAG_DIR/${README_STEM%.md}_${tag}.md"
  
  #  __init__.py files needed for import (create proper packages out of dirs)
  #+ tag is NOT part of filename
  for f in "${PACKAGE_FILES[@]}"; do
    touch_safe "$TAG_DIR/$f"
    # Info on the __init__.py helper
    echo "  ---------------------------------------------------------"
    echo "  First OS-agnostic Python helper, __init__.py"
    echo "  provided for tag, ``tag' at"
    echo "  '$TAG_DIR/$f'"
    echo "  to allow the directory name to serve as a package name,"
    echo "  simplifying imports."
    echo
  done

  # Notebooks with _<tag>.ipynb inside notebooks/
  for f in "${NB_FILES[@]}"; do
    base="$(basename -- "$f")"
    stem="${base%.*}"
    ext="${base##*.}"
    dst="$TAG_DIR/notebooks/${stem}_${tag}.${ext}"
    if [ "$WITH_NB_STUBS" = "1" ]; then
      [ -f "$dst" ] || emit_ipynb "$dst"
    else
      touch_safe "$dst"
    fi
  done

  # Python scripts with _<tag>.py
  for f in "${PY_FILES[@]}"; do
    base="$(basename -- "$f")"
    stem="${base%.*}"
    ext="${base##*.}"
    touch_safe "$TAG_DIR/scripts/${stem}_${tag}.${ext}"
  done

  # CMD placeholders with _<tag>.cmd
  for f in "${CMD_FILES[@]}"; do
    base="$(basename -- "$f")"
    stem="${base%.*}"
    ext="${base##*.}"
    touch_safe "$TAG_DIR/scripts/${stem}_${tag}.${ext}"
  done

  # PowerShell placeholders with _<tag>.ps1
  for f in "${THE_PS_FILES[@]}"; do
    base="$(basename -- "$f")"
    stem="${base%.*}"
    ext="${base##*.}"
    touch_safe "$TAG_DIR/scripts/${stem}_${tag}.${ext}"
  done
  
  # SH placeholders with _<tag>.cmd
  for f in "${THE_SH_FILES[@]}"; do
    base="$(basename -- "$f")"
    stem="${base%.*}"
    ext="${base##*.}"
    touch_safe "$TAG_DIR/scripts/${stem}_${tag}.${ext}"
  done
  
  # Untagged common helpers
  for f in "${UNTAGGED_COMMON_FILES}"; do
    touch_safe "$TAG_DIR/$f"
  done
  
  
  ##-----------------------------------------------------------------
  ##  I do include a couple of per-experiment helpers, written 
  ##+ in Python. The first is in case some kind of `touch' 
  ##+ functionality be desired that's consistent between Windows
  ##+ and Linux (*NIX). The second is an End Of Line Normalizer,
  ##+ a helper-function-version of programs like dos2unix / unix2dos
  ##+ This allows cross-platform creation of an experimental
  ##+ directory very quickly.
  
  ##  The idea of a `touch'-like function for PowerShell was
  ##+ abandoned. I _do_ include a per-experiment helper, written
  ##+ in Python, the "first" explained above.
  
  py_touch_path="$TAG_DIR/scripts/py_touch.py"
  if [ ! -f "$py_touch_path" ]; then
    cat << 'EOF' > "$py_touch_path"
import sys
from pathlib import Path
for f in sys.argv[1:]: Path(f).touch(exist_ok=True)
EOF

  fi
  
  # Info on the py_touch helper
  echo "  ---------------------------------------------------------"
  echo "  OS-agnostic helper,"
  echo "  $py_touch_path"
  echo "  provided for tag, ``$tag', in case it be desired."
  echo "  This could prove immensely helpful for Windows users."
  echo "  On *NIX, prefer touch(1) when available; this is a fallback."
  echo
  
  # Create (untagged-common) normalize_eol.py if missing
  norm_eol_path="$TAG_DIR/scripts/normalize_eol.py"
  if [ ! -f "$norm_eol_path" ]; then
    cat << 'EONormF' > "$norm_eol_path"
"""
Normalize line endings (EOL) for text files.

Usage modes:
  A) By extension map (recommended)
     python normalize_eol.py --root <DIR> --map sh=lf,ps1=crlf,cmd=crlf,py=lf,ipynb=lf,md=lf

  B) Explicit mode on listed files
     python normalize_eol.py --to-lf  file1 file2 ...
     python normalize_eol.py --to-crlf file1 file2 ...

Notes:
  - Skips binaries by a simple heuristic (NULL byte check).
  - Only rewrites when a change is needed.
"""

import argparse, os, sys
from pathlib import Path

def is_binary(data: bytes) -> bool:
  """
  Heuristic: treat as binary if there is a NUL byte.
  """
  return b"\x00" in data

def normalize_bytes(data: bytes, mode: str) -> bytes:
  """
  Convert EOLs:
    mode=''lf''   -> \n
    mode=''crlf'' -> \r\n
  """
  # First unify to LF
  text = data.replace(b"\r\n", b"\n").replace(b"\r", b"\n")
  if mode == "lf":
    return text
  elif mode == "crlf":
    return text.replace(b"\n", b"\r\n")
  else:
    raise ValueError(f"Unknown mode: {mode}")

def normalize_file(path: Path, mode: str) -> bool:
  """
  Normalize a single file in-place. Returns True if modified.
  """
  try:
    raw = path.read_bytes()
  except Exception:
    return False
  if is_binary(raw):
    return False
  new = normalize_bytes(raw, mode)
  if new != raw:
    path.write_bytes(new)
    return True
  return False

def parse_map(map_str: str) -> dict:
  """
  Parse ''ext=mode,ext=mode'' into dict like {''.sh'':''lf'', ''.ps1'':''crlf''}
  """
  out = {}
  for part in map_str.split(","):
    part = part.strip()
    if not part:
      continue
    k, v = part.split("=")
    ext = k.strip().lower()
    if not ext.startswith("."):
      ext = "." + ext
    out[ext] = v.strip().lower()
  return out

def normalize_by_map(root: Path, extmap: dict) -> int:
  """
  Walk root and apply per-extension modes. Returns count of modified files.
  """
  n = 0
  for p in root.rglob("*"):
    if not p.is_file():
      continue
    mode = extmap.get(p.suffix.lower())
    if not mode:
      continue
    if normalize_file(p, mode):
      n += 1
  return n

def main(argv=None):
  """
  CLI entry point.
  """
  ap = argparse.ArgumentParser()
  g = ap.add_mutually_exclusive_group()
  g.add_argument("--to-lf",   action="store_true", help="Force LF on listed files")
  g.add_argument("--to-crlf", action="store_true", help="Force CRLF on listed files")
  ap.add_argument("--root", type=Path, help="Directory to normalize recursively")
  ap.add_argument("--map",  type=str, help="Extension map like ''sh=lf,ps1=crlf''")
  ap.add_argument("files", nargs="*", type=Path, help="Files to normalize (with --to-*)")
  args = ap.parse_args(argv)

  # Mode B: explicit files
  if args.to_lf or args.to_crlf:
    mode = "lf" if args.to_lf else "crlf"
    changed = 0
    for f in args.files:
      if normalize_file(f, mode):
        changed += 1
    print(f"Changed {changed} files.")
    return 0

  # Mode A: by-extension map under --root
  if args.root and args.map:
    extmap = parse_map(args.map)
    changed = normalize_by_map(args.root, extmap)
    print(f"Changed {changed} files under {args.root}.")
    return 0

  ap.error("Provide either (--to-lf|--to-crlf files...) or --root DIR --map ext=mode,...")

if __name__ == "__main__":
  """
  Gets called if the module is called from command prompt, via
    e.g.
      > python normalize_eof.py <argument>
    OR
      $ python normalize_eof.py <argument>
  
  """
  
  #  Note that this next call combines calling main, the exiting with
  #+ its return value. I think it's even clearer than assigning the
  #+ return value of main to, say, retval, and then returning retval.
  
  sys.exit(main())

EONormF

  fi
  
  # Info on the normalize_eof helper
  echo "  ---------------------------------------------------------"
  echo "  OS-agnostic helper,"
  echo "  $norm_eol_path"
  echo "  provided for tag, '$tag'. IT WILL BE NECESSARY TO USE IT!"
  echo "  DO NOT MISS reading the pre-running instructions, in the $tag"
  echo "  README at Section: REQUIRED Pre-running Instructions, to allow"
  echo "  allow the files to run from your repo, whether on your local"
  echo "  machine or on any online/VPN/other machine."
  echo "  THIS IS NECESSARY WHETHER ON WINDOWS OR *NIX!"
  echo "  DO NOT PROCEED FURTHER WITHOUT FOLLOWING THOSE INSTRUCTIONS!"
  echo
  echo "  From bash, usage should be"
  echo "    \$ python \"\$TAG_DIR/scripts/normalize_eol.py \\\""
  echo "         --root \"\$TAG_DIR\" \\\""
  echo "         --map 'sh=lf,ps1=crlf,cmd=crlf,py=lf,ipynb=lf,md=lf' "
  echo
  echo "  From PowerShell, usage should be"
  echo "    PS> & python \"<path-to-tagdir>\\scripts\\normalize_eol.py\" --root <path-to-tagdir> --map \"sh=lf,ps1=crlf,cmd=crlf,py=lf,ipynb=lf,md=lf\""
  echo "  where <path-to-tagdir> is analogous to"
  echo "  \"$TAG_DIR\" in the Windows platform setup."
  echo 
  

  ##  .gitattributes content now added with ensure_gitattributes_entries
  ##+ function, where it is created/appended at the root of the git repo
  ##+ (if there is a git repo)
#b4#  # Create (untagged-common) .gitattributes (or append if already exists)
#b4#  gitattr_path="$TAG_DIR/.gitattributes"
#b4#  if [ ! -f "$gitattr_path" ]; then
#b4#    cat << 'EOGitAttrF' >> "$gitattr_path"
#b4#
#b4#    #  .gitattributes addition (or creation) for ease in using
#b4#    #+ platform-specific files (making it platform-agnostic)
#b4#*.sh              text eol=lf
#b4#*.ps1             text eol=crlf
#b4#*.cmd             text eol=crlf
#b4#*.py              text eol=lf
#b4#*.md              text eol=lf
#b4#*.ipynb           text eol=lf
#b4#.gitattributes    text eol=lf
#b4#
#b4#EOGitAttrF
#b4#
#b4#    # Info on the .gitattributes additions (creation)
#b4#    echo "  ---------------------------------------------------------"
#b4#    echo "  Helper to ensure correct EOL on OS-specific files,"
#b4#    echo "  $gitattr_path"
#b4#    echo "  provided for tag, '$tag'. This will allow everying"
#b4#    echo "  to be stress-free platform-agnostic. Only used"
#b4#    echo "  when things are done in a project where source"
#b4#    echo "  control is handled via Git, but it doesn't hurt to"
#b4#    echo "  have this in here."
#b4#    echo "  (Note that any files that should be excluded/ignored"
#b4#    echo "   are covered in the Git repo's root .gitignore file.)"
#b4#    echo
#b4#  
#b4#  fi

done

echo "--------------------------------------------------------------------"
echo "Project scaffolding with tags for files and tag-named subdirectories created at $ROOT_DIR"
echo "--------------------------------------------------------------------"
--------------------------------------------------

--------------------------------------------------
Script at:
/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_01/scripts/build_model_p_01.sh
----------
--------------------------------------------------

--------------------------------------------------
Script at:
/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_01/scripts/inference_p_01.sh
----------
--------------------------------------------------

--------------------------------------------------
Script at:
/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_01/scripts/train_model_p_01.sh
----------
--------------------------------------------------

--------------------------------------------------
Script at:
/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_03_e2e/scripts/build_model_p_03_e2e.sh
----------
--------------------------------------------------

--------------------------------------------------
Script at:
/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_03_e2e/scripts/inference_p_03_e2e.sh
----------
--------------------------------------------------

--------------------------------------------------
Script at:
/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_03_e2e/scripts/py_utils_p_03_e2e.sh
----------
--------------------------------------------------

--------------------------------------------------
Script at:
/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_03_e2e/scripts/train_model_p_03_e2e.sh
----------
--------------------------------------------------

=====..........................................................retval=0.........

(vanilla_cnn)  <=> conda environment, blank means none activated
[cifar10-vanilla-cnn] <=> git branch, blank means not in a git repo
bballdave025@MY-MACHINE ~/my_repos_dwb/fhtw-paper-code-prep
$
```

### Python scripts

```python
=====..........................................................retval=0.........

(vanilla_cnn)  <=> conda environment, blank means none activated
[cifar10-vanilla-cnn] <=> git branch, blank means not in a git repo
bballdave025@MY-MACHINE ~/my_repos_dwb/fhtw-paper-code-prep
$ find . -type f -iname "*.py" | sort | tr '\n' '\0' | xargs -I{} -0 bash -c 'orig={}; sixtydash="--------------------------------------------------"; tendash="----------"; echo "${sixtydash}"; echo "Script at:"; echo "$(realpath {})"; echo "${tendash}"; cat {}; echo "${sixtydash}"; echo;'
--------------------------------------------------
Script at:
/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/general_lab_notebooks_-_other_examples/notebook_cache_fallback_for_jupyter_cell.py
----------

# Notebook cache fallback (paste at very top)
If your notebook couldn’t see the fallback earlier, here’s the snippet again. Pop this in **before** importing TensorFlow:

```python
# --- CIFAR-10 cache + quieter TF ---
import os, shutil
from pathlib import Path

os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "2")
os.environ.setdefault("OMP_NUM_THREADS", "4")
os.environ.setdefault("TF_NUM_INTEROP_THREADS", "2")
os.environ.setdefault("TF_NUM_INTRAOP_THREADS", "4")

tagdir = Path(os.environ.get("TAGDIR", Path.cwd()))
if tagdir.name == "notebooks":
  tagdir = tagdir.parent
os.environ.setdefault("KERAS_HOME", str(tagdir))  # -> $TAGDIR/datasets
proj_cache = Path(os.environ["KERAS_HOME"]) / "datasets"
proj_cache.mkdir(parents=True, exist_ok=True)

home_cache = Path.home() / ".keras" / "datasets"
candidates = [("cifar-10-python.tar.gz", True), ("cifar-10-batches-py", False)]
if home_cache.exists() and not any((proj_cache / name).exists() for name, _ in candidates):
  for name, is_file in candidates:
      src = home_cache / name
      if src.exists():
          dst = proj_cache / src.name
          (shutil.copy2 if is_file else shutil.copytree)(src, dst, dirs_exist_ok=True)

--------------------------------------------------

--------------------------------------------------
Script at:
/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_01/__init__.py
----------
--------------------------------------------------

--------------------------------------------------
Script at:
/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_01/scripts/__init__.py
----------
--------------------------------------------------

--------------------------------------------------
Script at:
/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_01/scripts/normalize_eol.py
----------
"""
Normalize line endings (EOL) for text files.

Usage modes:
  A) By extension map (recommended)
     python normalize_eol.py --root <DIR> --map sh=lf,ps1=crlf,cmd=crlf,py=lf,ipynb=lf,md=lf

  B) Explicit mode on listed files
     python normalize_eol.py --to-lf  file1 file2 ...
     python normalize_eol.py --to-crlf file1 file2 ...

Notes:
  - Skips binaries by a simple heuristic (NULL byte check).
  - Only rewrites when a change is needed.
"""

import argparse, os, sys
from pathlib import Path

def is_binary(data: bytes) -> bool:
  """
  Heuristic: treat as binary if there is a NUL byte.
  """
  return b"\x00" in data

def normalize_bytes(data: bytes, mode: str) -> bytes:
  """
  Convert EOLs:
    mode=''lf''   -> \n
    mode=''crlf'' -> \r\n
  """
  # First unify to LF
  text = data.replace(b"\r\n", b"\n").replace(b"\r", b"\n")
  if mode == "lf":
    return text
  elif mode == "crlf":
    return text.replace(b"\n", b"\r\n")
  else:
    raise ValueError(f"Unknown mode: {mode}")

def normalize_file(path: Path, mode: str) -> bool:
  """
  Normalize a single file in-place. Returns True if modified.
  """
  try:
    raw = path.read_bytes()
  except Exception:
    return False
  if is_binary(raw):
    return False
  new = normalize_bytes(raw, mode)
  if new != raw:
    path.write_bytes(new)
    return True
  return False

def parse_map(map_str: str) -> dict:
  """
  Parse ''ext=mode,ext=mode'' into dict like {''.sh'':''lf'', ''.ps1'':''crlf''}
  """
  out = {}
  for part in map_str.split(","):
    part = part.strip()
    if not part:
      continue
    k, v = part.split("=")
    ext = k.strip().lower()
    if not ext.startswith("."):
      ext = "." + ext
    out[ext] = v.strip().lower()
  return out

def normalize_by_map(root: Path, extmap: dict) -> int:
  """
  Walk root and apply per-extension modes. Returns count of modified files.
  """
  n = 0
  for p in root.rglob("*"):
    if not p.is_file():
      continue
    mode = extmap.get(p.suffix.lower())
    if not mode:
      continue
    if normalize_file(p, mode):
      n += 1
  return n

def main(argv=None):
  """
  CLI entry point.
  """
  ap = argparse.ArgumentParser()
  g = ap.add_mutually_exclusive_group()
  g.add_argument("--to-lf",   action="store_true", help="Force LF on listed files")
  g.add_argument("--to-crlf", action="store_true", help="Force CRLF on listed files")
  ap.add_argument("--root", type=Path, help="Directory to normalize recursively")
  ap.add_argument("--map",  type=str, help="Extension map like ''sh=lf,ps1=crlf''")
  ap.add_argument("files", nargs="*", type=Path, help="Files to normalize (with --to-*)")
  args = ap.parse_args(argv)

  # Mode B: explicit files
  if args.to_lf or args.to_crlf:
    mode = "lf" if args.to_lf else "crlf"
    changed = 0
    for f in args.files:
      if normalize_file(f, mode):
        changed += 1
    print(f"Changed {changed} files.")
    return 0

  # Mode A: by-extension map under --root
  if args.root and args.map:
    extmap = parse_map(args.map)
    changed = normalize_by_map(args.root, extmap)
    print(f"Changed {changed} files under {args.root}.")
    return 0

  ap.error("Provide either (--to-lf|--to-crlf files...) or --root DIR --map ext=mode,...")

if __name__ == "__main__":
  """
  Gets called if the module is called from command prompt, via
    e.g.
      > python normalize_eof.py <argument>
    OR
      $ python normalize_eof.py <argument>
  
  """
  
  #  Note that this next call combines calling main, the exiting with
  #+ its return value. I think it's even clearer than assigning the
  #+ return value of main to, say, retval, and then returning retval.
  
  sys.exit(main())

--------------------------------------------------

--------------------------------------------------
Script at:
/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_01/scripts/py_build_model_p_01.py
----------
--------------------------------------------------

--------------------------------------------------
Script at:
/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_01/scripts/py_inference_p_01.py
----------
--------------------------------------------------

--------------------------------------------------
Script at:
/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_01/scripts/py_touch.py
----------
--------------------------------------------------

--------------------------------------------------
Script at:
/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_01/scripts/py_train_model_p_01.py
----------
--------------------------------------------------

--------------------------------------------------
Script at:
/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_01/scripts/py_utils_p_01.py
----------
"""
py_utils_p_01.py

Utility functions for experiment logging (Q&R acceptance).
"""

import json
from pathlib import Path
from datetime import datetime

# Default seed (can be overridden via env or caller)
SEED = 137

def _ts():
    """Return timestamp for filenames."""
    return datetime.now().strftime("%Y%m%dT%H%M%S")

def make_paths(seed: int = SEED, tagdir: str | Path | None = None):
    """
    Build file paths for CSV log and JSON test summary.

    Args:
        seed: random seed used for run
        tagdir: root of project (defaults to CWD or $TAGDIR)

    Returns:
        (csv_path, json_path)
    """
    tagdir = Path(tagdir or Path.cwd())
    outdir = tagdir / "outputs"
    csv_dir = outdir / "csv_logs"
    outdir.mkdir(parents=True, exist_ok=True)
    csv_dir.mkdir(parents=True, exist_ok=True)

    stamp = _ts()
    csv_path = csv_dir / f"train_history_seed{seed}_{stamp}.csv"
    json_path = outdir / f"test_summary_seed{seed}_{stamp}.json"
    return csv_path, json_path

def log_test_summary(test_acc: float,
                     loss: float | None = None,
                     seed: int = SEED,
                     tagdir: str | Path | None = None,
                     extra: dict | None = None):
    """
    Write test summary JSON and print the sentinel line.

    Args:
        test_acc: final test accuracy
        loss: optional test loss
        seed: random seed
        tagdir: project tag root
        extra: extra fields to include
    """
    _, json_path = make_paths(seed, tagdir)
    payload = {"seed": seed, "test_acc": float(test_acc)}
    if loss is not None:
        payload["test_loss"] = float(loss)
    if extra:
        payload.update(extra)

    json_path.parent.mkdir(parents=True, exist_ok=True)
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(payload, f, indent=2)

    print(f"[DONE] test_acc={payload['test_acc']:.4f}  ->  wrote {json_path}")
    return json_path

--------------------------------------------------

--------------------------------------------------
Script at:
/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_03_e2e/__init__.py
----------
--------------------------------------------------

--------------------------------------------------
Script at:
/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_03_e2e/scripts/__init__.py
----------
--------------------------------------------------

--------------------------------------------------
Script at:
/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_03_e2e/scripts/normalize_eol.py
----------
"""
Normalize line endings (EOL) for text files.

Usage modes:
  A) By extension map (recommended)
     python normalize_eol.py --root <DIR> --map sh=lf,ps1=crlf,cmd=crlf,py=lf,ipynb=lf,md=lf

  B) Explicit mode on listed files
     python normalize_eol.py --to-lf  file1 file2 ...
     python normalize_eol.py --to-crlf file1 file2 ...

Notes:
  - Skips binaries by a simple heuristic (NULL byte check).
  - Only rewrites when a change is needed.
"""

import argparse, os, sys
from pathlib import Path

def is_binary(data: bytes) -> bool:
  """
  Heuristic: treat as binary if there is a NUL byte.
  """
  return b"\x00" in data

def normalize_bytes(data: bytes, mode: str) -> bytes:
  """
  Convert EOLs:
    mode=''lf''   -> \n
    mode=''crlf'' -> \r\n
  """
  # First unify to LF
  text = data.replace(b"\r\n", b"\n").replace(b"\r", b"\n")
  if mode == "lf":
    return text
  elif mode == "crlf":
    return text.replace(b"\n", b"\r\n")
  else:
    raise ValueError(f"Unknown mode: {mode}")

def normalize_file(path: Path, mode: str) -> bool:
  """
  Normalize a single file in-place. Returns True if modified.
  """
  try:
    raw = path.read_bytes()
  except Exception:
    return False
  if is_binary(raw):
    return False
  new = normalize_bytes(raw, mode)
  if new != raw:
    path.write_bytes(new)
    return True
  return False

def parse_map(map_str: str) -> dict:
  """
  Parse ''ext=mode,ext=mode'' into dict like {''.sh'':''lf'', ''.ps1'':''crlf''}
  """
  out = {}
  for part in map_str.split(","):
    part = part.strip()
    if not part:
      continue
    k, v = part.split("=")
    ext = k.strip().lower()
    if not ext.startswith("."):
      ext = "." + ext
    out[ext] = v.strip().lower()
  return out

def normalize_by_map(root: Path, extmap: dict) -> int:
  """
  Walk root and apply per-extension modes. Returns count of modified files.
  """
  n = 0
  for p in root.rglob("*"):
    if not p.is_file():
      continue
    mode = extmap.get(p.suffix.lower())
    if not mode:
      continue
    if normalize_file(p, mode):
      n += 1
  return n

def main(argv=None):
  """
  CLI entry point.
  """
  ap = argparse.ArgumentParser()
  g = ap.add_mutually_exclusive_group()
  g.add_argument("--to-lf",   action="store_true", help="Force LF on listed files")
  g.add_argument("--to-crlf", action="store_true", help="Force CRLF on listed files")
  ap.add_argument("--root", type=Path, help="Directory to normalize recursively")
  ap.add_argument("--map",  type=str, help="Extension map like ''sh=lf,ps1=crlf''")
  ap.add_argument("files", nargs="*", type=Path, help="Files to normalize (with --to-*)")
  args = ap.parse_args(argv)

  # Mode B: explicit files
  if args.to_lf or args.to_crlf:
    mode = "lf" if args.to_lf else "crlf"
    changed = 0
    for f in args.files:
      if normalize_file(f, mode):
        changed += 1
    print(f"Changed {changed} files.")
    return 0

  # Mode A: by-extension map under --root
  if args.root and args.map:
    extmap = parse_map(args.map)
    changed = normalize_by_map(args.root, extmap)
    print(f"Changed {changed} files under {args.root}.")
    return 0

  ap.error("Provide either (--to-lf|--to-crlf files...) or --root DIR --map ext=mode,...")

if __name__ == "__main__":
  """
  Gets called if the module is called from command prompt, via
    e.g.
      > python normalize_eof.py <argument>
    OR
      $ python normalize_eof.py <argument>
  
  """
  
  #  Note that this next call combines calling main, the exiting with
  #+ its return value. I think it's even clearer than assigning the
  #+ return value of main to, say, retval, and then returning retval.
  
  sys.exit(main())

--------------------------------------------------

--------------------------------------------------
Script at:
/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_03_e2e/scripts/py_build_model_p_03_e2e.py
----------
--------------------------------------------------

--------------------------------------------------
Script at:
/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_03_e2e/scripts/py_inference_p_03_e2e.py
----------
--------------------------------------------------

--------------------------------------------------
Script at:
/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_03_e2e/scripts/py_touch.py
----------
--------------------------------------------------

--------------------------------------------------
Script at:
/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_03_e2e/scripts/py_train_model_p_03_e2e.py
----------
--------------------------------------------------

--------------------------------------------------
Script at:
/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/test_project_bash/p_03_e2e/scripts/py_utils_p_03_e2e.py
----------
--------------------------------------------------

--------------------------------------------------
Script at:
/home/bballdave025/my_repos_dwb/fhtw-paper-code-prep/validate_env.py
----------
import sys
import importlib
import psutil
from ptflops import get_model_complexity_info
from torchvision.models import resnet18

# -------------------------
# Package Checks, part 1
# -------------------------
# List of packages to check
packages = {
  "Python": sys.version,
  "tensorflow": "tensorflow",
  "torch": "torch",
  "torchvision": "torchvision",
  "torchaudio": "torchaudio",
  "numpy": "numpy",
  "pandas": "pandas",
  "scikit-learn": "sklearn",
  "opencv": "cv2",
  "Pillow": "PIL",
  "matplotlib": "matplotlib",
  "tensorboard": "tensorboard",
  "visualkeras": "visualkeras",
  "netron": "netron",
  "ptflops": "ptflops",
  "psutil": "psutil",
  "tqdm": "tqdm",
  "humanfriendly": "humanfriendly",
  "sagemaker": "sagemaker",
  "boto3": "boto3",
  "jupyterlab": "jupyterlab",
  "jsonschema": "jsonschema",
}

# -------------------------
# Package Checks, part 2
# -------------------------
def check_package(pkg_name, module_name):
  try:
    module = importlib.import_module(module_name)
    version = getattr(module, "__version__", "Unknown")
    print(f"[OK] {pkg_name} - version: {version}")
  except ImportError:
    print(f"[MISSING] {pkg_name} - not installed")
  finally:
    print(f"Finished check_package for {pkg_name} ({module_name})")
    print()
  ##endof:  try/except/finally <import and get attributes>
##endof:  def check_package


# -------------------------
# GPU Checks
# ------------------------
def check_gpu():
  print("\nGPU Check:")
  print("=" * 20)
    
    # TensorFlow GPU
  try:
    import tensorflow as tf
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
      print(f"[TF GPU] TensorFlow sees {len(gpus)} GPU(s): {[gpu.name for gpu in gpus]}")
    else:
      print("[TF GPU] No GPU detected for TensorFlow.")
  except ImportError:
    print("[TF GPU] TensorFlow not installed, cannot check GPU.")
  finally:
    print("Finished Tensorflow GPU part of check_gpu")
    print()
  ##endof:  try/except/finally <tensorflow gpu stuff>
  
    # PyTorch GPU
  try:
    import torch
    if torch.cuda.is_available():
      print(f"[Torch GPU] PyTorch sees {torch.cuda.device_count()} " + \
            f"GPU(s): {torch.cuda.get_device_name(0)}")
    else:
      print("[Torch GPU] No GPU detected for PyTorch.")
  except ImportError:
    print("[Torch GPU] PyTorch not installed, cannot check GPU.")
  finally:
    print("Finished PyTorch GPU part of check_gpu")
    print()
  ##endof:  try/except/finally <pytorch gpu stuff>
##endof:  def check_gpu

# -------------------------
# Memory and FLOPs Tests
# -------------------------
def check_memory_flops():
  print("\nMemory and FLOPs Test:")
  print("="*20)
  mem = psutil.virtual_memory()
  print(f"Total RAM: {mem.total/1e9:.2f} GB, Available: {mem.available/1e9:.2f} GB")
  # Small FLOPs test using ResNet18
  try:
    macs, params = get_model_complexity_info(
        resnet18(), 
        (3, 224, 224), 
        as_strings=True,
        print_per_layer_stat=False, 
        verbose=False
    )
    print(f"ResNet18 Params: {params}, FLOPs: {macs}")
  except Exception as e:
    print(f"ptflops test failed: {e}")
  finally:
    print("Finished check_memory_flops")
    print()
  ##endof:  try/except/finally <get_model_complexity_info>
##endof:  check_memory_flops

# -------------------------
# Main
# -------------------------
if __name__ == "__main__":
  '''
  Gets called when validate_env.py is called from the command line
  '''
  
  print("Environment Validation Report:")
  print("=" * 40)
  for name, mod in packages.items():
    if name == "Python":
      print(f"Python - version: {mod}")
    else:
      check_package(name, mod)
    ##endof:  if/else <python>
    
  check_gpu()
  check_memory_flops()
##endof:  if __name__ == "__main__"
--------------------------------------------------

=====..........................................................retval=0.........

(vanilla_cnn)  <=> conda environment, blank means none activated
[cifar10-vanilla-cnn] <=> git branch, blank means not in a git repo
bballdave025@MY-MACHINE ~/my_repos_dwb/fhtw-paper-code-prep
$

```

## After getting ready and going to some (i.e. a little bit of) church.

```bash
 <=> conda environment, blank means none activated
[cifar10-vanilla-cnn] <=> git branch, blank means not in a git repo
bballdave025@MY-MACHINE ~/my_repos_dwb/fhtw-paper-code-prep
$ type timestamp
timestamp is a function
timestamp () 
{ 
    date +"%s_%Y-%m-%dT%H%M%S%z"
}
=====..........................................................retval=0.........

 <=> conda environment, blank means none activated
[cifar10-vanilla-cnn] <=> git branch, blank means not in a git repo
bballdave025@MY-MACHINE ~/my_repos_dwb/fhtw-paper-code-prep
$ elapsed --check
9050
=====..........................................................retval=0.........

 <=> conda environment, blank means none activated
[cifar10-vanilla-cnn] <=> git branch, blank means not in a git repo
bballdave025@MY-MACHINE ~/my_repos_dwb/fhtw-paper-code-prep
$ echo "$(elapsed --check)/60" | bc -l
151.10000000000000000000
=====..........................................................retval=0.........

 <=> conda environment, blank means none activated
[cifar10-vanilla-cnn] <=> git branch, blank means not in a git repo
bballdave025@MY-MACHINE ~/my_repos_dwb/fhtw-paper-code-prep
$ type timestamp
timestamp is a function
timestamp () 
{ 
    date +"%s_%Y-%m-%dT%H%M%S%z"
}
=====..........................................................retval=0.........

 <=> conda environment, blank means none activated
[cifar10-vanilla-cnn] <=> git branch, blank means not in a git repo
bballdave025@MY-MACHINE ~/my_repos_dwb/fhtw-paper-code-prep
$ timestamp
1757264623_2025-09-07T110343-0600
=====..........................................................retval=0.........

 <=> conda environment, blank means none activated
[cifar10-vanilla-cnn] <=> git branch, blank means not in a git repo
bballdave025@MY-MACHINE ~/my_repos_dwb/fhtw-paper-code-prep
$ elapsed --help
Usage: elapsed [OPTION]

Options:
  -s, --start   Start the stopwatch
  -c, --check   Show elapsed time (in seconds, stopwatch keeps running)
  -p, --stop    Stop the stopwatch and show elapsed time (in seconds)
  -h, --help    Show this help message

If run without an option, this help will be displayed.
=====..........................................................retval=0.........

 <=> conda environment, blank means none activated
[cifar10-vanilla-cnn] <=> git branch, blank means not in a git repo
bballdave025@MY-MACHINE ~/my_repos_dwb/fhtw-paper-code-prep
$ elapsed -p
9098
=====..........................................................retval=0.........

 <=> conda environment, blank means none activated
[cifar10-vanilla-cnn] <=> git branch, blank means not in a git repo
bballdave025@MY-MACHINE ~/my_repos_dwb/fhtw-paper-code-prep
$ elapsed --check
No stopwatch running. Use --start to begin.
=====..........................................................retval=0.........

 <=> conda environment, blank means none activated
[cifar10-vanilla-cnn] <=> git branch, blank means not in a git repo
bballdave025@MY-MACHINE ~/my_repos_dwb/fhtw-paper-code-prep
$ timestamp
1757264660_2025-09-07T110420-0600
=====..........................................................retval=0.........

 <=> conda environment, blank means none activated
[cifar10-vanilla-cnn] <=> git branch, blank means not in a git repo
bballdave025@MY-MACHINE ~/my_repos_dwb/fhtw-paper-code-prep
$ exit
exit
Script done.
Raw log:   /home/bballdave025/work_logs/Lab_Notebook_bballdave025_1757257066_2025-09-07T085746-0600.log
Clean log: 
=====..........................................................retval=0.........

 <=> conda environment, blank means none activated
[cifar10-vanilla-cnn] <=> git branch, blank means not in a git repo
bballdave025@MY-MACHINE ~/my_repos_dwb/fhtw-paper-code-prep
$

```

In [19]:
!date +"%s_%Y-%m-%dT%H%M%S%z"

1757264785_2025-09-07T110625-0600
